In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_data= pd.read_csv("/kaggle/input/loan-prediction-dataset/train.csv")

train_data.sample(10)

In [ ]:
train_data.describe()

# Clean Data
Map non numeric values to numeric values


In [ ]:


train_data['Self_Employed'] = train_data.Self_Employed.map(dict(Yes=1, No=0))
train_data['Loan_Status'] = train_data.Loan_Status.map(dict(Y=1, N=0))
features = ["Gender", "Married", "Education", "Property_Area","Dependents"]
train_data = pd.concat([train_data,pd.get_dummies(train_data[features])], axis=1)
train_data.dropna(subset=['LoanAmount','Loan_Amount_Term','Credit_History'],inplace=True)
train_data.reset_index(drop=True, inplace=True)
train_data

In [ ]:
# Standardize values
x  = train_data[['ApplicantIncome','CoapplicantIncome','LoanAmount','Loan_Amount_Term']] #A & S [1,2]
print ("\nOriginal data values : \n", x)
from sklearn import preprocessing 
 
min_max_scaler = preprocessing.MinMaxScaler(feature_range =(0, 1))  
# Scaled feature 
x_after_min_max_scaler = min_max_scaler.fit_transform(x) 
  
print ("After min max Scaling : \n", x_after_min_max_scaler) 
loan_std = pd.DataFrame(x_after_min_max_scaler, columns =['ApplicantIncome_Std','CoapplicantIncome_Std','LoanAmount_Std','Loan_Amount_Term_Std'])
train_data = pd.concat([train_data,loan_std], axis=1) 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Loan_Status"]
print(train_data.columns)
features = [ 
       'Self_Employed',  'Credit_History', 
       'Gender_Female', 'Gender_Male', 'Married_No', 'Married_Yes',
       'ApplicantIncome_Std',
       'CoapplicantIncome_Std', 'LoanAmount_Std', 'Loan_Amount_Term_Std']
X = train_data[features]

from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data_with_imputed_values = my_imputer.fit_transform(X)

X = pd.DataFrame(data_with_imputed_values,columns = features)

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X)

df = pd.DataFrame({'Actual': y, 'Predicted': predictions})
df['match?'] = np.where(df['Actual'] == df['Predicted'], 1,0)
(df['match?'].sum() / df['match?'].count() * 100)

# 83.36483931947069





 